# Simple RAG system for CSV files

In this notebook, we will build a RAG system that processes and queries CSV documents. We will use OpenAI embeddings and FAISS for efficient similarity search. The system will encode the CSV document content into a vector store, split the content into manageable chunks, and retrieve relevant information based on user queries.


In [1]:
from dotenv import load_dotenv
import os
import pandas as pd
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
import faiss

from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

# Load environment variables from .env file
load_dotenv()

# Access the API key
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

### Load the CSV file
We will load the CSV file into a pandas DataFrame. This dataset serves as the source of information that the RAG system will query.

In [2]:
file_path = 'customers-100.csv'
data = pd.read_csv(file_path)

# Preview the first few rows of the dataset
data.head()

,Index,Customer Id,First Name,Last Name,Company,City,Country,Phone 1,Phone 2,Email,Subscription Date,Website
0,1,DD37Cf93aecA6Dc,Sheryl,Baxter,Rasmussen Group,East Leonard,Chile,229.077.5154,397.884.0519x718,zunigavanessa@smith.info,2020-08-24,http://www.stephenson.com/
1,2,1Ef7b82A4CAAD10,Preston,Lozano,Vega-Gentry,East Jimmychester,Djibouti,5153435776,686-620-1820x944,vmata@colon.com,2021-04-23,http://www.hobbs.com/
2,3,6F94879bDAfE5a6,Roy,Berry,Murillo-Perry,Isabelborough,Antigua and Barbuda,+1-539-402-0259,(496)978-3969x58947,beckycarr@hogan.com,2020-03-25,http://www.lawrence.com/
3,4,5Cef8BFA16c5e3c,Linda,Olsen,"Dominguez, Mcmillan and Donovan",Bensonview,Dominican Republic,001-808-617-6467x12895,+1-813-324-8756,stanleyblackwell@benson.org,2020-06-02,http://www.good-lyons.com/
4,5,053d585Ab6b3159,Joanna,Bender,"Martin, Lang and Andrade",West Priscilla,Slovakia (Slovak Republic),001-234-203-0635x76146,001-199-446-3860x3486,colinalvarado@miles.net,2021-04-17,https://goodwin-ingram.com/


The CSV file contains customer data with attributes such as first name, last name, company, etc. The goal is to use this data in a Q&A system that can provide information about customers based on specific queries.

### Load and split the CSV data into chunks
Now, we will load the CSV file into the `CSVLoader` from LangChain. This loader provides a specialized way to split the content into chunks suitable for embedding and retrieval tasks.

- Each row from the CSV is treated as a separate "document" containing the data in that row. This means each row becomes an individual chunk. For example, if the CSV has multiple columns (e.g., `first_name`, `last_name`, `company`), each row is stored as a separate document containing the information from those columns, ensuring that our RAG system can work with the data as if it were a natural text document.
- If the document is too large (for example, if there are large text columns in the CSV), it may be split into smaller chunks. This chunking ensures that when we generate embeddings, the text is of a manageable size, improving the performance of the embedding model and the search system. In our case, no explicit chunking based on text size happens here.

In [3]:
loader = CSVLoader(file_path=file_path)

# Load and split the CSV file into chunks
docs = loader.load_and_split()

### Create embeddings for the chunks
Next, we create embeddings for the chunks using OpenAI API. We use OpenAI's embeddings to convert the text into vectors. This allows us to capture semantic meaning.

##### Initialize embedding model
First, we will initialize `OpenAIEmbeddings` which will be used to generate vector embeddings for each document.

In [4]:
embeddings = OpenAIEmbeddings()

##### Create a FAISS index
Now, we use FAISS to create an index that will allow for fast similarity search. It creates a flat (non-compressed) index using L2 distance (Euclidean distance) as the metric for comparing vectors.

In [5]:
# Create a FAISS index for storing and querying the embeddings
index = faiss.IndexFlatL2(len(OpenAIEmbeddings().embed_query(" ")))

It gets as input the size of the embedding vector. This is done by calling `embed_query(" ")` on the `OpenAIEmbeddings()` object, which generates an embedding for a dummy query (a space `" "`). The length of the resulting vector is the size that the FAISS index will expect for each of the embeddings. It ensures that the index is properly set up to handle embeddings of the correct size.

#####  Create the FAISS vector store
This creates a FAISS vector store, which will manage both the document embeddings and the FAISS index. It provides a way to efficiently store, index, and search the embeddings.

In [6]:
# Create a FAISS vector store
vector_store = FAISS(
    embedding_function=OpenAIEmbeddings(),
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

- **`embedding_function=OpenAIEmbeddings()`**: This tells the FAISS vector store how to embed new documents. In our case, it will use the `OpenAIEmbeddings` object, which means it will generate OpenAI embeddings when required.
- **`index=index`**: The previously created FAISS index is passed here. This index will store the embeddings, and during a search, it will find the closest (most similar) embeddings to the query.
- **`docstore=InMemoryDocstore()`**: The `InMemoryDocstore()` is a simple in-memory document store. It will hold the documents that correspond to the embeddings in the vector store. This allows the system to retrieve the actual document content when we perform a search. In this case, it’s an in-memory store, which means it will not persist data after the script ends. We can use other types of document stores (like a database) if persistence is needed.
- **`index_to_docstore_id={}`**: This is an empty dictionary, which is used to map each FAISS index entry to a specific document. It helps link a particular embedding (vector) back to the actual document in the document store (`docstore`). Wi will need to populate this dictionary with mappings between the index and documents, but in the current code, it is empty for simplicity.

##### Add documents (embeddings) to the vector store
Now, we will add the documents (in the form of their embeddings) to the FAISS vector store.

In [7]:
# Add the documents to the vector store
vector_store.add_documents(documents=docs)

['fe11d5a3-5030-45b5-a600-f95ef2047f78',
 '949db4de-26df-4a47-b21d-5dc19f756d36',
 '12890ec2-0a5a-4637-9a28-f650fc3a7296',
 'ae0e854b-efae-4bc4-a298-58821c21843e',
 'c142918b-f13c-47e0-82bd-39e90150c1f1',
 '6206d751-5675-485a-ab49-0e3ea3545ff7',
 'e0bbb04d-b4de-4709-af71-a91223a1b748',
 '36209884-1ab0-4b32-bae4-3c30652dd869',
 '71c44ea6-0397-4fc8-8b55-bb646be66c4e',
 'b6963c1c-f1fb-4d7b-910c-4499d99415c8',
 '754e58fb-d0c0-4364-bea3-bd727980739c',
 'cbcd12d4-c2ad-41af-9a64-4f0c0da01b9a',
 '10d35cc5-fc3a-4559-9a2a-c64d17801ab4',
 'd8cc634e-97d2-42e5-a795-48403fee7761',
 'e9f553c2-3da4-4535-8882-7019a69be99f',
 'b5ad1bd8-5d12-49de-bf4e-a55eb7cb2c0a',
 '07e175a8-9ba7-4273-867a-d9cc5bd2e7b0',
 '4e3507f2-f807-41c6-ad74-9a366c84ea0e',
 '9a215089-197e-424d-8880-43b831f6101f',
 '1464164d-9e09-4ddd-a531-68ee27340d7f',
 '92209a57-6342-4de4-91b9-ff5c6c5b6eea',
 '45f80d87-5768-4f4a-a16d-00a1fd2b4d5d',
 'e4d027f2-8d5d-4420-a9ef-000e9b1524a9',
 '652341b7-74d0-4700-832f-bf2f0998c885',
 '51cd0a44-86c4-

What does this line do?
1. **Embedding generation**: The `add_documents` method takes the list of documents (`docs`) and processes each document's content. For each document, the content is passed through the embedding function (`OpenAIEmbeddings()`) and converted into a high-dimensional numerical vector.
2. **Storing embeddings in FAISS**: Once the embeddings are generated for each document, they are stored in the FAISS vector store. The vector store acts as a database where embeddings are indexed and can later be searched for similarity.
3. **Vector store structure**: The vector store consists of two main components:
     - The embeddings themselves, which are the high-dimensional vectors generated from the document content.
     - The index, which helps FAISS to quickly perform similarity searches and retrieve the most relevant documents based on a query.
   - When the documents are added to the vector store, they are linked to their corresponding embeddings in the FAISS index, allowing for fast and efficient retrieval later on.

### Set up the retriever
The retriever is responsible for fetching the most relevant documents when a query is issued. We configure the retriever to use the FAISS vector store we just created. This means that the vector store, which holds the embeddings and the documents, will now have the capability to search for documents that are relevant to a given query.

In [8]:
retriever = vector_store.as_retriever()

### Set up the system prompt
The system prompt defines how the LLM should respond to user queries. In this case, it instructs the LLM to use the context retrieved from the vector store to answer questions concisely. Then, we will use `ChatPromptTemplate` to create a structured chat prompt template. It allows us to specify how the conversation will flow and how the LLM should respond to both system and human (user) messages.

The `system` and `human` keywords play important roles in controlling the flow and context of the interaction.
  - **System message**: The first part of the tuple `("system", system_prompt)` indicates that the `system_prompt` will be used to define the behavior and instructions for the LLM. It is not directly user input but an internal instruction for the LLM's performance. For example, it guides the model's behavior for the entire conversation or it defines the role of the assistant (e.g., "You are a helpful assistant for answering questions based on retrieved context").
  - **Human message**: The second part `("human", "{input}")` defines the input that will be provided by the user. This could be any question or statement the user is asking the assistant. The `{input}` is a placeholder for the user’s query.

In [9]:
# Set up system prompt
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
    
])

The `{context}` placeholder in the prompt is where the retrieved document content (the relevant chunks of text) will be inserted. The LLM will use this content to form its answers to the user’s query.

### Create the question-answering chain and the RAG chain
We now create two chains using Langchain, that handle the question-answering process: the question-answer chain and the RAG chain. The purpose of these chains is to tie together the LLM, the retrieved context, and the user query.

In [10]:
# Create an instance of the OpenAI Chat API
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

# Create the question-answer chain
question_answer_chain = create_stuff_documents_chain(llm, prompt)

# Set up the RAG chain with the retriever and question-answer chain
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

The `create_stuff_documents_chain` creates a question-answering chain that combines the LLM with a prompt template.
- **LLM**: This is the language model that will generate the answers.
- **Prompt template**: This is the structured template (defined earlier) that instructs the model on how to answer and holds the actual query that the user asks.

The `create_retrieval_chain` creates a RAG chain that connects the retriever (which fetches relevant context from the vector store) with the question-answer chain (which generates the final answer). By combining the retriever and question-answer chain, we create a RAG pipeline.
- **Retriever**: This part searches the vector store (which contains embeddings of the documents) and retrieves the most relevant documents based on the query.
- **Question-answer chain**: Once the context is retrieved, this chain processes both the retrieved context and the query to generate a response.

### Test the system
We will now query the RAG system by providing a user query and obtaining an answer based on the documents stored in the vector store. The process involves invoking the RAG chain to fetch relevant context and generate a response.

This input is populated into the human part of the prompt template defined earlier. Processing the query:
  - The retriever in the RAG chain will search the vector store for relevant documents related to the query.
  - These documents, or chunks of context, are then passed to the question-answer chain, where the LLM uses the context and the query to generate an answer.

In [11]:
# Test query to ask the assistant
answer= rag_chain.invoke({"input": "which company does sheryl Baxter work for?"})

# Display the answer
answer['answer']

'Sheryl Baxter works for Rasmussen Group.'